Importing libraries and several well log dataset to combine. This process is necesarry due to: not every lithology are avaiable on every well logs. so to deterimine all the lithology value several well logs data are combined. 

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('31_2-7.csv')
df2 = pd.read_csv('35_11-10.csv')
df3 = pd.read_csv('34_10-19.csv')
df4 = pd.read_csv('33_9-17.csv')
df5 = pd.read_csv('25_11-24.csv')
df6 = pd.read_csv('16_10-2.csv')
df6 = df6.iloc[1:] 
# Drop the first row, due to some of the dataset have the first row as physics unit such as ms/cm etc
df7 = pd.read_csv('15_9-14.csv')
df7 = df7.iloc[1:]
df8 = pd.read_csv('7_1-1.csv')
df8.head()

There's some extreme negative unit on 'RXO' columns, in well logs extreme negative value like -999.99 usually detirmine that the value is missing. so extreme negative value is replaced to NaN or missing data 

In [ ]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)
df = df.apply(pd.to_numeric, errors='coerce')
df['RXO'] = df['RXO'].replace(-597.3141479, np.nan)
df['RXO'].unique()



Due to hardware computing capabilities, columns is reduced to make the dataframe smaller. we chose the most crusial property on lithology classification like 'GR', 'RHOB', 'NPHI', 'DTC', 'RDEP', 'RXO', 'RSHA'

In [ ]:
filtered_model = ['GR', 'RHOB', 'NPHI', 'DTC', 'RDEP', 'RXO', 'RSHA', 'FORCE_2020_LITHOFACIES_LITHOLOGY']
df_filtered_model = df[filtered_model]
df_filtered_model.info()

Now for Handling missing value and scaling it using StandardScaler, we make a model to predict and imputate the missing value. Model used on this imputatation process is RandomForestReggresor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Separate features and target columns
target_columns = ['GR', 'RHOB', 'NPHI', 'DTC', 'RDEP', 'RXO', 'RSHA']
features_columns = [col for col in df_filtered_model.columns if col not in target_columns]

# Create a DataFrame to store imputed values
df_imputed = df_filtered_model.copy()

# Initialize scaler
scaler = StandardScaler()

# Separate complete and incomplete data
complete_data = df_filtered_model.dropna(subset=target_columns)
incomplete_data = df_filtered_model[df_filtered_model.isnull().any(axis=1)]

# Scale the features
scaler.fit(complete_data[features_columns])
complete_data_scaled = pd.DataFrame(scaler.transform(complete_data[features_columns]), columns=features_columns, index=complete_data.index)
incomplete_data_scaled = pd.DataFrame(scaler.transform(incomplete_data[features_columns]), columns=features_columns, index=incomplete_data.index)

# Impute missing values column by column
for column in target_columns:
    # Create a mask for missing values in the column
    mask = df_filtered_model[column].isnull()
    
    if mask.any():
        print(f"Imputing missing values for column: {column}")
        
        # Prepare training data from the complete dataset
        X_train = complete_data_scaled
        y_train = complete_data[column]
        
        # Initialize and train the RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators=100, random_state=42)
        regressor.fit(X_train, y_train)
        
        # Predict missing values in the incomplete data
        X_missing = incomplete_data_scaled[mask]
        imputed_values = regressor.predict(X_missing)
        
        # Impute missing values in the DataFrame
        df_imputed.loc[mask, column] = imputed_values
        
        # Optionally print out the performance
        if not X_train.empty:
            y_pred_train = regressor.predict(X_train)
            mse = mean_squared_error(y_train, y_pred_train)
            print(f"Imputation MSE for column '{column}': {mse:.4f}")

# Verify the imputation
print(df_imputed.info())


Now we evaluate the model accuracy or ecceftiveness, using mean squared error and r2score. then we loop the evalutaion to each model for each columns

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

target_columns = ['GR', 'RHOB', 'NPHI', 'DTC', 'RDEP', 'RXO', 'RSHA']

# Iterate over each target column
for target_column in target_columns:
    # Separate features and target variable
    features = df_filtered_model.drop(columns=[target_column])
    target = df_filtered_model[target_column]

    # Remove rows with NaN in target column
    valid_data = df_filtered_model.dropna(subset=[target_column])
    features_valid = valid_data.drop(columns=[target_column])
    target_valid = valid_data[target_column]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features_valid, target_valid, test_size=0.3, random_state=42)

    # Initialize and train the RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    regressor.fit(X_train, y_train)

    # Predict on the test set
    y_pred = regressor.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Evaluation for column '{target_column}':")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R-squared (R²): {r2:.4f}")

    # Save the model
    model_filename = f'{target_column}_regressor.pkl'
    joblib.dump(regressor, model_filename)
    print(f"Model for column '{target_column}' saved as '{model_filename}'\n")


Evaluation for column 'GR':
Mean Squared Error (MSE): 237.3588
R-squared (R²): 0.8145
Model for column 'GR' saved as 'GR_regressor.pkl'

Evaluation for column 'RHOB':
Mean Squared Error (MSE): 0.0035
R-squared (R²): 0.9260
Model for column 'RHOB' saved as 'RHOB_regressor.pkl'

Evaluation for column 'NPHI':
Mean Squared Error (MSE): 0.0010
R-squared (R²): 0.9286
Model for column 'NPHI' saved as 'NPHI_regressor.pkl'

Evaluation for column 'DTC':
Mean Squared Error (MSE): 56.2338
R-squared (R²): 0.9380
Model for column 'DTC' saved as 'DTC_regressor.pkl'

Evaluation for column 'RDEP':
Mean Squared Error (MSE): 309.5248
R-squared (R²): 0.3007
Model for column 'RDEP' saved as 'RDEP_regressor.pkl'

Evaluation for column 'RXO':
Mean Squared Error (MSE): 376.4542
R-squared (R²): 0.9643
Model for column 'RXO' saved as 'RXO_regressor.pkl'

Evaluation for column 'RSHA':
Mean Squared Error (MSE): 73.1303
R-squared (R²): 0.6105
Model for column 'RSHA' saved as 'RSHA_regressor.pkl'



The model has been evaluated and have great evaltion, then we import the model and test it for future use

In [ ]:
models = {
    'GR': joblib.load('GR_regressor.pkl'),
    'RHOB': joblib.load('RHOB_regressor.pkl'),
    'NPHI': joblib.load('NPHI_regressor.pkl'),
    'DTC': joblib.load('DTC_regressor.pkl'),
    'RDEP': joblib.load('RDEP_regressor.pkl'),
    'RXO': joblib.load('RXO_regressor.pkl'),
    'RSHA': joblib.load('RSHA_regressor.pkl')
}

def impute_column(df, column_name, model):
    missing_mask = df[column_name].isna()
    if missing_mask.sum() > 0:  # Check if there are missing values to impute
        missing_data = df[missing_mask]
        if missing_data.shape[0] > 0:
            features = missing_data.drop(columns=[column_name])
            # Ensure that features do not contain NaNs
            features = features.fillna(features.median())
            predicted_values = model.predict(features)
            df.loc[missing_mask, column_name] = predicted_values
        else:
            print(f"No missing data found for column '{column_name}'.")
    else:
        print(f"No missing values to impute for column '{column_name}'.")

# Impute missing values
for column in models.keys():
    if column in df_filtered_model.columns:
        impute_column(df_filtered_model, column, models[column])

# Print the results
print("First few rows of the imputed DataFrame:")
print(df_filtered_model.head())

print("\nSummary statistics of the DataFrame:")
print(df_filtered_model.describe())

print("\nCount of missing values in each column:")
print(df_filtered_model.isna().sum())

# Save the DataFrame to CSV
df_filtered_model.to_csv('df_filtered_model_imputed.csv', index=False)
print("\nDataFrame with imputed values saved to 'df_filtered_model_imputed.csv'.")
